<a href="https://colab.research.google.com/github/Kat-ies/Bio_BSU/blob/main/lab_3/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab_3 : Smith-Waterman algorithm**
*Ekaterina Yurkovskaya, 3rd group*



In [ ]:
!pip3 install Bio

# **Algorithm.**

In [18]:
import numpy as np


def get_i_j_mismatch(first_amino_acid, second_amino_acid,mismatch_penalty, substitution_matrix=None):
    if substitution_matrix is None:
        return -mismatch_penalty if  first_amino_acid == second_amino_acid else mismatch_penalty
    else:
        return substitution_matrix[first_amino_acid][second_amino_acid]



def aligment(ref_seq, seq, deletion_penalty=-1, insertion_penalty=-1, mismatch_penalty=-1, substitution_matrix=None):
    dp = np.zeros((len(ref_seq) + 1, len(seq) + 1))

    for i in range(len(ref_seq)+1):
        dp[i][0] = i * deletion_penalty

    for j in range(len(seq)+1):
        dp[0][j] = j * insertion_penalty

     
    start_i, start_j = len(ref_seq), len(seq)
    start_score = -1


    for i in range(1, len(ref_seq)+1):
        for j in range(1, len(seq)+1):
            dp[i][j] = max(0,
                           dp[i - 1][j] + deletion_penalty,
                           dp[i][j - 1] + insertion_penalty,
                           dp[i - 1][j - 1] + get_i_j_mismatch(ref_seq[i-1], seq[j-1],
                                                               mismatch_penalty, substitution_matrix))
            if dp[i][j] > start_score:
                start_score = dp[i][j]
                start_i, start_j = i, j


    # print(dp)

    queue = [(start_i, start_j, "", "")]
    seq1, seq2 = [], []
    
 
    while queue:
        i, j, current_seq1, current_seq2 = queue.pop()

        if dp[i][j] == 0:
            seq1.append(current_seq1)
            seq2.append(current_seq2)

        else:
            if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + \
            get_i_j_mismatch(ref_seq[i - 1], seq[j - 1],mismatch_penalty, substitution_matrix):
                queue.append((i - 1, j - 1, current_seq1 + ref_seq[i-1], current_seq2 + seq[j-1]))
            if i > 0 and dp[i][j] == dp[i - 1][j] + deletion_penalty:
                queue.append((i - 1, j, current_seq1 + ref_seq[i-1],current_seq2 + '-'))
            if j > 0 and dp[i][j] == dp[i][j - 1] + insertion_penalty:
                queue.append((i, j - 1, current_seq1 + '-',current_seq2 + seq[j - 1]))


    for i in range(len(seq1)):
        print('First sequency:  ' + seq1[i][::-1])
        print('Second sequency: ' + seq2[i][::-1])
        print('_______________________\n' )

# **Tests.**

In [4]:
from Bio.Align import substitution_matrices

blosum = substitution_matrices.load('BLOSUM62')
pam = substitution_matrices.load('PAM250')

In [20]:
aligment('TGTTACGG', 'GGTTGACTA', deletion_penalty=-2, insertion_penalty=-2, mismatch_penalty=-3)

First sequency:  GTT-AC
Second sequency: GTTGAC
_______________________



In [19]:
 aligment('CCAGAGAC', 'CCAAGGCT')

First sequency:  CCAGAG
Second sequency: CCA-AG
_______________________



In [21]:
aligment('ALPAQGPWKSPISYFRMTCVSLRVNHRPQFIEAWGNR','AAPVQEGPWTISSBMQSLDHBVBQFLIEAWNR', substitution_matrix = blosum,
         deletion_penalty=-5, insertion_penalty=-5)


First sequency:  ALPAQ-GPWKSPISYFRMTCVSLRVNHRP-QF-IEAWGNR
Second sequency: AAPVQEGPW-T-ISS-BMQ--SL--DHBVBQFLIEAW-NR
_______________________

First sequency:  ALPAQ-GPWKSPISYFRMTCVSLRVNHRP-QF-IEAWGNR
Second sequency: AAPVQEGPW-T-IS-SBMQ--SL--DHBVBQFLIEAW-NR
_______________________

First sequency:  ALPAQ-GPWKSPISYFRMTCVSLRVNHR-PQF-IEAWGNR
Second sequency: AAPVQEGPW-T-ISS-BMQ--SL--DHBVBQFLIEAW-NR
_______________________

First sequency:  ALPAQ-GPWKSPISYFRMTCVSLRVNHR-PQF-IEAWGNR
Second sequency: AAPVQEGPW-T-IS-SBMQ--SL--DHBVBQFLIEAW-NR
_______________________



In [22]:
aligment('ALPAQGPWKSPISYFRMTCVSLRVNHRPQFIEAWGNR','AAPVQEGPWTISSBMQSLDHBVBQFLIEAWNR', substitution_matrix = pam,
         deletion_penalty=-5, insertion_penalty=-5)

First sequency:  PAQ-GPWKSPISYFRMTCVSLRVNHRPQF-IEAWGNR
Second sequency: PVQEGPW-T-ISS-BMQ--SLD-HBVBQFLIEAW-NR
_______________________

First sequency:  PAQ-GPWKSPISYFRMTCVSLRVNHRPQF-IEAWGNR
Second sequency: PVQEGPW-T-IS-SBMQ--SLD-HBVBQFLIEAW-NR
_______________________

